# Pandas API on Spark

## Przygotowania

Na początku pobierz dane, które będziemy przetwarzali 

In [1]:
%%sh
wget https://jankiewicz.pl/bigdata/bigdata-sp/netflix.zip

--2024-12-06 10:10:53--  https://jankiewicz.pl/bigdata/bigdata-sp/netflix.zip
Resolving jankiewicz.pl (jankiewicz.pl)... 109.95.159.1
Connecting to jankiewicz.pl (jankiewicz.pl)|109.95.159.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77010721 (73M) [application/zip]
Saving to: ‘netflix.zip.4’

     0K .......... .......... .......... .......... ..........  0% 1.16M 63s
    50K .......... .......... .......... .......... ..........  0% 2.35M 47s
   100K .......... .......... .......... .......... ..........  0%  208M 32s
   150K .......... .......... .......... .......... ..........  0% 2.36M 31s
   200K .......... .......... .......... .......... ..........  0%  178M 25s
   250K .......... .......... .......... .......... ..........  0%  201M 21s
   300K .......... .......... .......... .......... ..........  0%  202M 18s
   350K .......... .......... .......... .......... ..........  0% 2.45M 20s
   400K .......... .......... .......... .......... ......

Następnie je rozpakuj 

In [2]:
%%sh
unzip -o netflix.zip

Archive:  netflix.zip
  inflating: movie_titles.csv        
  inflating: rates/part-00.csv       
  inflating: rates/part-01.csv       
  inflating: rates/part-02.csv       
  inflating: rates/part-03.csv       
  inflating: rates/part-04.csv       
  inflating: rates/part-05.csv       
  inflating: rates/part-06.csv       
  inflating: rates/part-07.csv       
  inflating: rates/part-08.csv       
  inflating: rates/part-09.csv       
  inflating: rates/part-10.csv       
  inflating: rates/part-11.csv       


Załaduj je do systemu plików HDFS

In [3]:
%%sh
hadoop fs -put -f movie_titles.csv /tmp/
hadoop fs -put -f rates /tmp/

Sprawdź czy są dostępne w ich źródłowych (dla naszego przetwarzania) miejscach 

In [4]:
%%sh
hadoop fs -ls /tmp

Found 4 items
drwxrwxrwt   - hdfs hadoop          0 2024-12-06 09:05 /tmp/hadoop-yarn
drwx-wx-wx   - hive hadoop          0 2024-12-06 09:06 /tmp/hive
-rw-r--r--   2 root hadoop     577561 2024-12-06 10:11 /tmp/movie_titles.csv
drwxr-xr-x   - root hadoop          0 2024-12-06 10:11 /tmp/rates


Podglądnij jak wygląda ich budowa 

In [5]:
%%sh
hadoop fs -head /tmp/movie_titles.csv

ID,Year,Title
1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW
6,1997,Sick
7,1992,8 Man
8,2004,What the #$*! Do We Know!?
9,1991,Class of Nuke 'Em High 2
10,2001,Fighter
11,1999,Full Frame: Documentary Shorts
12,1947,My Favorite Brunette
13,2003,Lord of the Rings: The Return of the King: Extended Edition: Bonus Material
14,1982,Nature: Antarctica
15,1988,Neil Diamond: Greatest Hits Live
16,1996,Screamers
17,2005,7 Seconds
18,1994,Immortal Beloved
19,2000,By Dawn's Early Light
20,1972,Seeta Aur Geeta
21,2002,Strange Relations
22,2000,Chump Change
23,2001,Clifford: Clifford Saves the Day! / Clifford's Fluffiest Friend Cleo
24,1981,My Bloody Valentine
25,1997,Inspector Morse 31: Death Is Now My Neighbour
26,2004,Never Die Alone
27,1962,Sesame Street: Elmo's World: The Street We Live On
28,2002,Lilo and Stitch
29,2001,Boycott
30,2003,Something's Gotta Give
31,1999,Classic Albums: Meat Loaf: Bat Out

In [6]:
%%sh
hadoop fs -head /tmp/rates/part-00.csv

date,film_id,user_id,rate
1999-11-11,1367,510180,5
1999-11-11,1798,510180,5
1999-11-11,2866,510180,3
1999-11-11,3730,510180,4
1999-11-11,3870,510180,2
1999-11-11,5571,510180,4
1999-11-11,5625,510180,4
1999-11-11,6615,510180,5
1999-11-11,8079,510180,2
1999-11-11,8357,510180,4
1999-11-11,8651,510180,2
1999-11-11,9003,510180,3
1999-11-11,9392,510180,3
1999-11-11,9798,510180,3
1999-11-11,10341,510180,4
1999-11-11,10774,510180,3
1999-11-11,11234,510180,5
1999-11-11,11313,510180,2
1999-11-11,12470,510180,2
1999-11-11,12473,510180,5
1999-11-11,12818,510180,2
1999-11-11,14660,510180,2
1999-11-11,15057,510180,5
1999-11-11,15105,510180,4
1999-11-11,15894,510180,3
1999-11-11,16465,510180,3
1999-11-11,17064,510180,2
1999-11-11,17764,510180,5
1999-12-06,2478,510180,3
1999-12-06,2948,510180,3
1999-12-06,3421,510180,3
1999-12-06,5474,510180,2
1999-12-06,6902,510180,4
1999-12-06,9536,510180,5
1999-12-06,11080,510180,3
1999-12-06,11259,510180,3
1999-12-06,11612,510180,3
1999-12-06,14455,510180,3
1999-1

## Odczyt danych

Na początek kontekst

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Ustawmy stosowne zmienne środowiskowe, a następnie zaimportujmy potrzebne biblioteki

In [8]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [9]:
import pyspark.pandas as ps

Zaczytajmy dane do *Pandas Dataframe on Spark*

Na początku informacje na temat filmów

In [10]:
movies_ps = ps.read_csv('/tmp/movie_titles.csv')

/usr/lib/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Dowiedzmy się jak wygląda "budowa" naszych danych

In [11]:
movies_ps.shape

(17770, 3)

In [12]:
movies_ps.columns

Index(['ID', 'Year', 'Title'], dtype='object')

In [13]:
movies_ps.head()

,ID,Year,Title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


Czas zatem na oceny

In [14]:
rates_ps = ps.read_csv('/tmp/rates')

/usr/lib/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Dowiedzmy się jak wygląda "budowa" naszych danych

In [15]:
rates_ps.shape

(14780972, 4)

In [16]:
rates_ps.columns

Index(['date', 'film_id', 'user_id', 'rate'], dtype='object')

In [17]:
rates_ps.head()

,date,film_id,user_id,rate
0,2003-04-07,1256,406841,4
1,2003-04-07,1256,79527,4
2,2003-04-07,1256,108859,4
3,2003-04-07,1260,1816643,3
4,2003-04-07,1260,1827071,3


To co, jesteśmy w domu? Zatem do dzieła. Trzy proste zadania

# Zadania

## Zadanie 1

Określ ile filmów nie uzyskało żadnej oceny. Utwórz *Pandas Dataframe on Spark*, który będzie zawierał komplet informacji filmów ograniczonych tylko do tych, które mają chociaż jedną ocenę.  

In [18]:
# Grupowanie ocen po filmach, aby sprawdzić liczbę ocen dla każdego filmu
film_ratings_count = rates_ps.groupby("film_id").size().reset_index(name="rating_count")

# Łączenie danych o filmach z licznikami ocen
movies_with_ratings = movies_ps.merge(
    film_ratings_count,
    how="left",
    left_on="ID",
    right_on="film_id"
)

# Zamiana brakujących wartości (brak ocen) na 0
movies_with_ratings["rating_count"] = movies_with_ratings["rating_count"].fillna(0)

# Wyznaczanie liczby filmów bez ocen
no_ratings_count = movies_with_ratings[movies_with_ratings["rating_count"] == 0].shape[0]
print(f"Liczba filmów bez żadnej oceny: {no_ratings_count}")

# Utworzenie DataFrame tylko dla filmów, które mają co najmniej jedną ocenę
movies_with_at_least_one_rating = movies_with_ratings[movies_with_ratings["rating_count"] > 0]

# Wyświetlenie pierwszych kilku wierszy wynikowego DataFrame
movies_with_at_least_one_rating.head()


Liczba filmów bez żadnej oceny: 6056


,ID,Year,Title,film_id,rating_count
0,148,2001,Sweet November,148,5469
1,463,1962,The Twilight Zone: Vol. 12,463,588
2,471,1931,City Lights,471,48
4,833,1996,White Squall,833,1558
5,1088,1980,Hammer House of Horror,1088,36


## Zadanie 2

Dla każdej dekady premiery filmów utwórz dwa rankingi

- trzech filmów o największej liczbie ocen
- trzech filmów o największej średniej ocen

In [19]:
# Konwersja kolumn na odpowiednie typy danych
movies_ps["Year"] = movies_ps["Year"].astype(int)
rates_ps["rate"] = rates_ps["rate"].astype(float)

# Dodanie kolumny z dekadą premiery filmu
movies_ps["Decade"] = (movies_ps["Year"] // 10) * 10

# Łączenie danych o filmach i ocenach
movies_with_rates = rates_ps.merge(
    movies_ps,
    how="inner",
    left_on="film_id",
    right_on="ID"
)

# Grupowanie i obliczenia
movie_stats = movies_with_rates.groupby(["Decade", "ID", "Title"]).agg(
    num_ratings=("rate", "count"),   # Liczba ocen
    avg_rating=("rate", "mean")     # Średnia ocen
).reset_index()

# Top 3 filmy z największą liczbą ocen dla każdej dekady
top3_most_rated_per_decade = movie_stats.sort_values(
    by=["Decade", "num_ratings"], ascending=[True, False]
).groupby("Decade").head(3).reset_index(drop=True)

print("Top 3 filmy z największą liczbą ocen dla każdej dekady:")
print(top3_most_rated_per_decade)

# Top 3 filmy z najwyższą średnią ocen dla każdej dekady
top3_highest_avg_rating_per_decade = movie_stats.sort_values(
    by=["Decade", "avg_rating"], ascending=[True, False]
).groupby("Decade").head(3).reset_index(drop=True)

print("\nTop 3 filmy z najwyższą średnią ocen dla każdej dekady:")
print(top3_highest_avg_rating_per_decade)


Top 3 filmy z największą liczbą ocen dla każdej dekady:


    Decade     ID                                        Title  num_ratings  avg_rating
0   1890.0   7654                Lumiere Brothers' First Films           30    3.700000
1   1900.0   4975  D.W. Griffith: Years of Discovery 1909-1913            8    2.875000
2   1910.0   8821                        The Birth of a Nation          611    2.929624
3   1910.0    394                                           20          489    3.312883
4   1910.0   8539                  The Cabinet of Dr. Caligari          457    3.575492
5   1920.0   2847                            The Mark of Zorro         1951    3.182983
6   1920.0  13798                     The Last of the Mohicans         1347    3.668894
7   1920.0  17335                                   Metropolis          756    3.855820
8   1930.0   3605        The Wizard of Oz: Collector's Edition        18318    4.088110
9   1930.0   7416      Gone with the Wind: Collector's Edition        12098    3.923789
10  1930.0  15129               

    Decade     ID                                        Title  num_ratings  avg_rating
0   1890.0   7654                Lumiere Brothers' First Films           30    3.700000
1   1900.0   4975  D.W. Griffith: Years of Discovery 1909-1913            8    2.875000
2   1910.0   8539                  The Cabinet of Dr. Caligari          457    3.575492
3   1910.0   3387                                  Intolerance          128    3.406250
4   1910.0    394                                           20          489    3.312883
5   1920.0  15016                   The Passion of Joan of Arc          262    4.011450
6   1920.0   5231                         The General (Silent)          669    3.878924
7   1920.0   4249                          Battleship Potemkin          484    3.863636
8   1930.0   1129                         Mutiny on the Bounty            1    5.000000
9   1930.0  12235                                 Libeled Lady            1    5.000000
10  1930.0   2796      Imitation

## Zadanie 3

Dla trzech wybranych użytkowników, wśród tych, którzy ocenili co najemniej 10 filmów, wyznacz trzy najbardziej rekomendowane filmy do obejrzenia. 

Wskaż źródła opisujące zastosowane przez Ciebie podejście. 

Koniecznie uzupełnij kod szczegółowymi komentarzami. 

In [20]:
import pyspark.pandas as ps

# Konwersja kolumn na odpowiednie typy danych
rates_ps["rate"] = rates_ps["rate"].astype(float)

# Grupowanie ocen przez użytkowników i filmy, liczymy średnią ocenę oraz liczbę ocen
user_movie_ratings = rates_ps.groupby(["user_id", "film_id"]).agg(
    avg_rating=("rate", "mean"),
    num_ratings=("rate", "count")
).reset_index()

# Wybieramy użytkowników, którzy ocenili co najmniej 10 filmów
user_ratings_count = user_movie_ratings.groupby("user_id")["num_ratings"].sum()
users_with_min_ratings = user_ratings_count[user_ratings_count >= 10]

# Wybieramy 3 pierwszych użytkowników
selected_users = users_with_min_ratings.head(3).index.tolist()

# Dla każdego wybranego użytkownika, obliczamy 3 najbardziej rekomendowane filmy
recommended_movies = []
for user_id in selected_users:
    # Filtrujemy dane dla danego użytkownika
    user_ratings = user_movie_ratings[user_movie_ratings["user_id"] == user_id]
    
    # Sortujemy filmy według średniej oceny (od najwyższej) i bierzemy 3 najlepsze
    top_movies = user_ratings.nlargest(3, "avg_rating")
    
    # Dodajemy kolumnę 'user_id', by śledzić, który użytkownik
    top_movies['user_id'] = user_id
    
    # Dodajemy wynik do listy rekomendacji
    recommended_movies.append(top_movies[["user_id", "film_id", "avg_rating"]])

# Łączenie wyników w jedną tabelę
recommended_movies_df = ps.concat(recommended_movies).reset_index(drop=True)

# Połączenie z tabelą 'movies_ps' na podstawie 'film_id' w celu dodania tytułu filmu
final_recommended_df = recommended_movies_df.merge(
    movies_ps[['ID', 'Title']],
    how='left',
    left_on='film_id',
    right_on='ID'
)

# Usuwamy zbędną kolumnę 'ID' z wyników
final_recommended_df = final_recommended_df[['user_id', 'Title', 'avg_rating']]

# Wyświetlenie końcowego DataFrame z rekomendacjami
print("Trzy najbardziej rekomendowane filmy dla wybranych użytkowników:")
print(final_recommended_df)


/usr/lib/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Trzy najbardziej rekomendowane filmy dla wybranych użytkowników:


   user_id                          Title  avg_rating
0   376576  National Lampoon's Van Wilder         5.0
1   376576                          Seven         5.0
2   376576            Babylon 5: Season 2         5.0
3   335555                     Braveheart         5.0
4   335555            The Tuskegee Airmen         5.0
5   335555                         Selena         5.0
6   210744            The Prince of Egypt         5.0
7   210744                 The Terminator         5.0
8   210744                          U-571         5.0
